<a href="https://colab.research.google.com/github/Lesiazhou/QM2/blob/main/PDF_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Jan  2 16:22:25 2025

@author: dianecomon
"""

from PIL import Image, ImageFont, ImageDraw
import numpy as np
import camelot
import requests
import PyPDF2
import re

# Parameters
path_PDFs = '/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/' # path on own computer (i.e. local path)
list_URL = ['http://www.cawater-info.net/aral/data/pdf/aral_1911-1985_en.pdf', 'http://www.cawater-info.net/aral/data/pdf/aral_1986-2006_en.pdf', 'http://www.cawater-info.net/aral/data/pdf/aral_2007-2018_en.pdf']
list_csv = ['water_1911_%d.csv', 'water_1986_%d.csv', 'water_2007_%d.csv'] # local files i want to scrape
path_csv = '/Users/dianecomon/Desktop/UCL/BASC0005 QM2/Datasets/Clean/' # where i want to save new csv


# Search inside a PDF to find a keyword, and return the page where it is
def find_page_with_string(file_PDF, search_strings):
   page_num = -1
   x = PyPDF2.PdfFileReader(file_PDF)
   n_pages = x.getNumPages()
   for i in range(0, n_pages):
       page_text = x.getPage(i).extractText()
       page_text = " ".join(page_text.split())
       for s in search_strings:
           if re.search(s,page_text):
                page_num = i
                break
   return page_num

# Download PDFs
def download_PDFs(dict_URLs, path_PDFs):
   dict_local_files = {}
   for y in dict_URLs:
       response = requests.get(dict_URLs[y])
       output_name = path_PDFs + str(y) + '.pdf'
       with open(output_name, 'wb') as f:
           f.write(response.content)
       dict_local_files[y] = output_name
   return dict_local_files

#local_files = download_PDFs(list_URL, path_PDFs)

#year = 2022
#file_name = local_files[year]

#page_n = find_page_with_string(file_name, ['Table 2', 'Таблица 2'])


# Generic function to convert strings to numbers
def convert_to_number(s):
   if isinstance(s, (int, float)):
       return float(s)
   elif isinstance(s, str):
       if all([c in(['0','1','2','3','4','5','6','7','8','9',',','.',' ']) for c in s]):
           if sum([c in(['0','1','2','3','4','5','6','7','8','9']) for c in s]) == 0:
               # No digits - do not convert
               return s
           elif s.find('.')<0 and s.find(',')<0:
               # No decimals
               return float(s.replace(' ',''))
           elif s.find('.')<0 and s.find(',')>=0:
               # There are ',' and no '.'. Assume ',' means decimal separator
               return float(s.replace(' ','').replace(',','.'))
           elif s.find('.')>=0 and s.find(',')<0:
               # There are '.' and no ','. Assume '.' means decimal separator
               # But if there are several '.' do not convert
               if s.find('.') != s.rfind('.'):
                   return s
               else:
                   return float(s.replace(' ',''))
           else:
               # There are both '.' and ','. Assume '.' means decimal separator
               return float(s.replace(' ','').replace(',',''))
       else:
           # The string contains non-numerical characters - do not convert it
           return s
   else:
       print('Warning - class %s not recognised' % str(type(s)))


# Apply the conversion function to the content of the Pandas extracted from the PDF
for i,URL in enumerate(list_URL):
    tables = camelot.read_pdf(URL, pages='1') # URLs used only have 1 page
    for j in range(len(tables)):
        b = tables[j].df
        for row_num, row_name in enumerate(b.index):
           for col_num, col_name in enumerate(b.columns):
               b.iloc[row_num,col_num] = convert_to_number(b.iloc[row_num,col_num])

        b.to_csv(path_csv + list_csv[i] %j)

